**Group 11**

**Simplex Method**

In [1]:
import numpy as np

def simplex(c, A, b):

    num_vars = len(c)
    num_constraints = len(b)

    # Create the initial tableau
    tableau = np.hstack((A, np.eye(num_constraints), b.reshape(-1, 1)))
    tableau = np.vstack((tableau, np.hstack((-c, np.zeros(num_constraints + 1)))))

    # Simplex iterations
    while True:
        # Identify entering variable (most negative coefficient in the objective row)
        entering = np.argmin(tableau[-1, :-1])
        if tableau[-1, entering] >= 0:
            # Optimal solution found
            break

        # Identify leaving variable (minimum ratio test)
        ratios = tableau[:-1, -1] / tableau[:-1, entering]
        ratios[ratios < 0] = np.inf  # Ignore negative ratios
        leaving = np.argmin(ratios)

        # Pivot operation
        pivot = tableau[leaving, entering]
        tableau[leaving] /= pivot  # Normalize pivot row

        for i in range(len(tableau)):
            if i != leaving:
                tableau[i] -= tableau[i, entering] * tableau[leaving]

    # Extract the solution
    optimal_values = np.zeros(num_vars)
    for i in range(num_vars):
        col = tableau[:-1, i]
        if np.count_nonzero(col) == 1 and np.any(col == 1):
            row = np.where(col == 1)[0][0]
            optimal_values[i] = tableau[row, -1]

    obj_value = tableau[-1, -1]

    return tableau, optimal_values, obj_value

**Dual Simplex**

In [2]:
import numpy as np
def dual_simplex(c, A, b):
    # Number of variables and constraints
    num_vars = len(c)
    num_constraints = len(b)

    # Create an initial tableau (A x <= b)
    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))

    # #maximize with <= constraints
    # c = -c

    # Fill the tableau
    tableau[:num_constraints, :num_vars] = A  # Coefficients of constraints
    tableau[:num_constraints, num_vars:num_vars + num_constraints] = np.eye(num_constraints)  # Slack variables
    tableau[:num_constraints, -1] = b  # Right-hand side (RHS)
    tableau[-1, :num_vars] = c

    # Dual Simplex iterations
    while True:
        # Step 1: Check if tableau is feasible (all RHS >= 0)
        if np.all(tableau[:-1, -1] >= 0):
            # Feasible solution found
            break

        # Step 2: Choose the leaving variable (most negative entry in RHS)
        pivot_row = np.argmin(tableau[:-1, -1])

        # Step 3: Choose the entering variable (pivot column based on smallest ratio in objective row)
        row = tableau[pivot_row, :-1]
        valid_cols = np.where(row < 0)[0]  # Valid columns for entering variable
        if len(valid_cols) == 0:
            raise ValueError("No valid pivot column found. Problem might be unbounded.")

        # Step 4: Ratio test to select entering variable
        ratios = tableau[-1, valid_cols] / row[valid_cols]
        pivot_col = valid_cols[np.argmin(ratios)]

        # Step 5: Pivot to make pivot element 1 and others in the column 0
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

    # Extracting the final optimal values for the original variables
    optimal_values = np.zeros(num_vars)
    for i in range(num_vars):
        col = tableau[:-1, i]
        if np.count_nonzero(col) == 1 and np.any(col == 1):
            row = np.where(col == 1)[0][0]
            optimal_values[i] = tableau[row, -1]

    # Final tableau and optimal values
    return tableau, optimal_values, tableau[-1, -1]

**Gomory Cut**

In [3]:
def gomory_cut(tableau):
    num_vars = len(c)  # Number of original variables (excluding slack and objective)

    fractional_rows = []

    for i in range(num_vars):
        # Check if the row corresponds to a basic variable (only one non-zero entry)
        non_zero_indices = np.nonzero(tableau[:, i])[0]
        if len(non_zero_indices) == 1:  # Ensure there's exactly one non-zero entry
            row_idx = non_zero_indices[0]
            if tableau[row_idx, -1] != np.floor(tableau[row_idx, -1]):  # Check if the solution is fractional
                fractional_rows.append(row_idx)

    if len(fractional_rows) == 0:
        print("No fractional rows found. The solution is already integer.")
        return None, None

    # Choose the first fractional basic variable (row)
    row_idx = fractional_rows[0]

    # Get the fractional part of the chosen row
    row = tableau[row_idx, :-1]
    rhs_value = tableau[row_idx, -1]
    rhs_fractional_part = rhs_value - np.floor(rhs_value)

    # Construct the Gomory cut (new constraint)
    new_cut = np.array([element - np.floor(element) if element >= 0 else -(np.floor(element) - element) for element in row])
    new_rhs = -rhs_fractional_part

    # Add slack variable: Append 1 to the new cut row
    new_cut_row = np.append(-new_cut, [1])  # Slack variable for the new cut
    new_cut_row = np.append(new_cut_row, [new_rhs])  # RHS for the new cut

    # Expand the tableau with an additional slack column and new row
    new_tableau = np.hstack([tableau[:, :-1], np.zeros((tableau.shape[0], 1)), tableau[:, -1:]])  # Add slack column
    new_tableau = np.vstack([new_tableau[:-1, :], new_cut_row, new_tableau[-1, :]])  # Insert new cut row before objective

    return new_tableau, new_cut_row

Dual simplex but takes table as input

In [4]:
def dual_simplex_tableau(tableau):
  num_vars = len(c)
  while True:
        # Step 1: Check if tableau is feasible (all RHS >= 0)
        if np.all(tableau[:-1, -1] >= 0):
            # Feasible solution found
            break

        # Step 2: Choose the leaving variable (most negative entry in RHS)
        pivot_row = np.argmin(tableau[:-1, -1])

        # Step 3: Choose the entering variable (pivot column based on smallest ratio in objective row)
        row = tableau[pivot_row, :-1]
        valid_cols = np.where(row < 0)[0]  # Valid columns for entering variable
        if len(valid_cols) == 0:
            raise ValueError("No valid pivot column found. Problem might be unbounded.")

        # Step 4: Ratio test to select entering variable
        ratios = tableau[-1, valid_cols] / row[valid_cols]
        pivot_col = valid_cols[np.argmin(ratios)]

        # Step 5: Pivot to make pivot element 1 and others in the column 0
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

    # Extracting the final optimal values for the original variables
  optimal_values = np.zeros(num_vars)
  for i in range(num_vars):
      col = tableau[:-1, i]
      if np.count_nonzero(col) == 1 and np.any(col == 1):
          row = np.where(col == 1)[0][0]
          optimal_values[i] = tableau[row, -1]

    # Final tableau and optimal values
  return tableau, optimal_values, tableau[-1, -1]

**Logic**

  1. Use simplex or dual_simplex function (whichever appropriate) to get the initial optimal tableau
  2. Pass the tableau to add_gomory_cut function to get the new tableau which has new constraints added to it.
  3. The new tableau is passed to dual_simplex_tableau function to get new table, if the solution is not integer, repeat from step 2.

**Main Code**

In [5]:
def is_integer_solution(solution, tolerance=1e-5):
    return np.all(np.abs(solution - np.round(solution)) < tolerance)

In [6]:
def cutting_plane_method(c, A, b):

    # Step 1: Solve the linear relaxation using simplex / dual simplex

    if any(element < 0 for element in b):
      tableau, solution, obj_value = dual_simplex(c, A, b)
      tableau[-1,:] = -tableau[-1,:]
      print(solution)
    else:
      tableau, solution, obj_value = simplex(c, A, b)
      print(solution)

    i = 0

    while not is_integer_solution(solution):
        # Step 2: Generate Gomory cut if solution is not integer
        tableau, cut = gomory_cut(tableau)
        if i%2 == 0:
          tableau[-1,:] = -tableau[-1,:]

        i += 1

        # Step 3: Re-solve the modified LP using dual simplex
        tableau, solution, obj_value = dual_simplex_tableau(tableau)
        print(solution)

    obj_value = np.dot(c, solution)

    return solution, obj_value

**DEMO**

---


Problem: max c.T*x subject to Ax <= b
1. c should be of maximize form
2. constraints should be of Ax <= b form

In [7]:
c = [-4, -3, -5]  # Coefficients for the objective function (maximize) z = c.T*x
A = [[-2,-3,-1],[-1,-2,-4],[-3,-1,-2],[1,0,1],[0,1,2]]  # Coefficient matrix for inequalities Ax <= b
b = [-10,-15,-12,7,6]  # Right-hand side of inequalities (Ax <= b)

cutting_plane_method(c,A,b)

[3.  0.4 2.8]
[3.         0.66666667 2.66666667]
[3. 2. 2.]


(array([3., 2., 2.]), -28.000000000000007)